# Data Journalism Lesson 8: Data Smells

Learn how to spot potential issues in your data before they lead to incorrect conclusions. Stop a wrong story before it starts.

In [ ]:
# Setup code for the notebook
import pandas as pd
from IPython.display import display, HTML

# Helper functions for grading exercises
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_glimpse(df_info_output):
    # Simple check if info() was called (it prints to stdout, doesn't return easily)
    # In a real scenario, might capture stdout or check specific properties
    # For this exercise, we'll assume if the code runs without error, it's likely correct
    # A placeholder check:
    if df_info_output is None: # info() returns None
         display_feedback(True, 'Good job using info() to inspect the DataFrame!', '')
    else:
         display_feedback(False, '', 'Make sure you are calling the .info() method on your DataFrame.')

def check_tally(result_df, expected_col, expected_rows_min=1):
    correct = False
    message = ''
    if isinstance(result_df, pd.DataFrame) and expected_col in result_df.columns and 'n' in result_df.columns:
        if len(result_df) >= expected_rows_min:
            correct = True
            message = f'Correctly tallied the counts for the {expected_col} column!'
        else:
            message = f'The tally for {expected_col} seems too short. Did you group correctly?'
    elif isinstance(result_df, pd.Series): # Handle value_counts() output
         if result_df.name == expected_col or expected_col in result_df.index.name:
            if len(result_df) >= expected_rows_min:
                correct = True
                message = f'Correctly tallied the counts for the {expected_col} column using value_counts()!'
            else:
                message = f'The tally for {expected_col} seems too short.'
         else:
            message = f'Are you counting values for the correct column ({expected_col})?'
    else:
        message = f'Expected a DataFrame (from groupby/size/reset_index) or a Series (from value_counts()) for the tally of {expected_col}. Check your methods.'
    display_feedback(correct, message, message)

def check_read_csv(df, expected_rows):
    correct = isinstance(df, pd.DataFrame) and len(df) == expected_rows
    display_feedback(correct, 'Great work! You imported your states plane crashes.', 'Check the file path or the read_csv function.')

In [ ]:
state = "Minnesota"

dataurl = f"../_static/plane-crashes/{state.lower()}.csv"

crashes = pd.read_csv(dataurl)
nrows = len(crashes)
ncolumns = len(crashes.columns)

countyLanguage = ('county', 'counties')

In [ ]:
# Glue variables for use in markdown
from myst_nb import glue

glue("state_full_name", state, display=False)
glue("nrows", nrows, display=False)
glue("county_singular", countyLanguage[0], display=False)
glue("county_plural", countyLanguage[1], display=False)

## The Goal

In this lesson, you'll learn about the critical concept of "data smells" - common issues and inconsistencies that can arise in datasets. By the end of this tutorial, you'll understand how to conduct an initial assessment of a dataset to identify potential problems like missing data, wrong data types, gaps in data, and internal inconsistencies. You'll practice using Python functions like `.info()`, `.groupby()`, and `.size()` (or `.value_counts()`) to explore your data systematically. This skill is essential for data journalists to ensure the accuracy and reliability of their analyses before drawing any conclusions.

## What is Data Journalism?

Chad Day, the chief elections analyst at the Associated Press, starts every data analysis he does with what amounts to a fight.

"I think that I always look at the data as it's going to try to trick me. And I have to find all of the tricks that it's going to throw at me," he said. "And then I play to the strengths, right? Because you're going to identify a bunch of weaknesses that are in the data. And you don't want to base your analysis on those weaknesses unless your story is about the weaknesses in the data."

So what does he -- and many other data analysts -- do? You've heard of giving something the smell test, yes? This is the very nerdy equivalent of the smell test. We call them data smells.

"Every single time we use data, we have to do the basic checks. And that means that we run those basic things every single time," Day said. "And it should be part of your muscle memory."

What's a data smell test?

"Let's use an example," Day said. "It's 50 state data. I make sure that all 50 states are in there, right? I make sure that if there are additional-- like if it's 51 or 52, is DC included?"

Any time you are given a dataset from anyone, you should immediately be suspicious. Is this data what I think it is? Does it include what I expect? Is there anything I need to know about it? Will it produce the information I expect?

Failure to give data the smell test [can lead you to miss stories and get your butt kicked on a competitive story](https://source.opennews.org/en-US/learning/handling-data-about-race-and-ethnicity/).

Giving your data the smell test isn't new. Exploratory Data Analysis or EDA has been around for a long time -- John Tukey started pushing the idea in 1970 and wrote the text that would lay the groundwork for generations of data analysts in 1977 (I have a copy in my office!). But EDA treats data in an almost neutral way - it's almost naive. Data smells treat data with skepticism and distrust. With data smells, we're trying to find common mistakes in data because we have to know if they are there. [For more on data smells, read the GitHub wiki post that started it all](https://github.com/nikeiubel/data-smells/wiki/Ensuring-Accuracy-in-Data-Journalism). The common mistakes we're looking for are:

-   Missing data
-   Gaps in data
-   Wrong type of data
-   Outliers
-   Sharp curves
-   Conflicting information within a dataset
-   Conflicting information across datasets
-   Wrongly derived data
-   Internal inconsistency
-   External inconsistency
-   Wrong spatial data
-   Unusable data, including non-standard abbreviations, ambiguous data, extraneous data, inconsistent data

Not all of these data smells are detectable in code. You may have to ask people about the data. You may have to compare it to another dataset yourself. Does the agency that uses the data produce reports from the data? Does your analysis match those reports? That will expose wrongly derived data, or wrong units, or mistakes you made with inclusion or exclusion.

But many of them are, and they are easy, knowing what you already know. You just need a little code, a dollop of logic and a healthy skepticism that the data you have is good.

## The Basics

Every dataset has quirks that make certain kinds of analysis annoyingly difficult. For example, the State of Nebraska publishes a dataset of every currently incarcerated person in the state prison system. A question that might come up in news stories -- how many people are currently serving prison time for certain crimes? For example: Meth related crimes. The problem? There's several dozen unique ways that the state has recorded possession of methamphetamine. Different spellings, different abbreviations, meth for short, methamphetamine spelled out, odd ways of abbreviating possession. It would take months of work to fix all the different ways the state has recorded a meth-related crime. And would it be worth doing that work for a single sentence of a story? Not likely.

But some can be fixed quicker. But you won't know what needs fixed -- or even if your data is worth using -- until you give your data the smell test.

Let's work on some examples the NTSB plane crash data, which we used in the last exercise.

First we'll need pandas. Your first step is always importing libraries and you'll need to run this step in nearly every single thing you do.

In [ ]:
import pandas as pd

Now import the data.

In [ ]:
crashes_df = pd.read_csv("../_static/plane-crashes/minnesota.csv")

Now, let's take a look at where this data has flaws.

## Wrong Type

First, let's look at the **Wrong Type Of Data** problem. We can sniff that out by looking at the output of `.info()`.

### Exercise 1: Using .info() for more than column names

Use the `.info()` method on the `crashes_df` DataFrame to see the column names, non-null counts, and data types.

In [ ]:
crashes_df.____()

First things first: What type of data are all of the columns being imported as? It's important to look at columns you know you are going to use. Or look at columns that -- just looking at their name -- suggest they should be a specific type of data.

In this data, for example, is the `ReportNo` column a number? ReportNo is short for Report Number, so it suggests it should be? But, recall your basics -- if you don't plan to do math on it, then it's not a number. In this case, the column is empty, so the imported data type might be `float64` (pandas often uses float for columns with NAs, even if they seem like integers) or `object` if it was completely empty or mixed.

Another example: `FatalInjuryCount` or `SeriousInjuryCount`. Is a count a number? It sure is. If you wanted to know how many people died in plane crashes in your state, you'd want to sum those counts. You can't sum them if they aren't numbers. Are these counts numbers? Check the `Dtype` column in the `.info()` output.

One place we can pretend there's a problem -- look at the `first_bad_date` column. What type is that? Compare it to the `EventDate` column. The `EventDate` column might be `datetime64[ns]` if pandas parsed it correctly, or it might be `object` (string). That means Python/pandas knows that column is a date and a time together, and will treat it as such if it's a datetime type. Our `first_bad_date` column is likely `object`. Is that a date type? No. Would pandas recognize that as a date without help? Maybe, maybe not (as we saw in the previous lesson).

## Missing Data

The second smell we can find in code is **missing data**. We can do that through a series of Group By and Count steps. In aggregates, we explicitly used `groupby()` and `agg()` or `size()`. Here, because this is exploratory, we can use the `.value_counts()` method as a shortcut for single columns, or `groupby().size()`.

Let's first look at the `HighestInjuryLevel` column. Is that column consistently filled out?

### Exercise 2: Missing data part 1

Use `.value_counts()` on the `HighestInjuryLevel` column to see the different values and their counts. Pay attention to `NaN` (Not a Number), which is pandas' way of representing missing values.

In [ ]:
# Get counts of each unique value in 'HighestInjuryLevel'
# Use dropna=False to include NaN counts
injury_counts = crashes_df[____].value_counts(dropna=False)
print(injury_counts)

# Check the exercise
check_tally(injury_counts, 'HighestInjuryLevel')

What we're looking for here are answers that don't make sense. One you're looking for are blanks (which might appear as empty strings `''` if they exist, though less common than `NaN`). Another: `NaN`. In many states, there are crashes that have a `HighestInjuryLevel` of None and also `NaN`. What is the difference? Are they the same thing? Are `NaN` cases where the NTSB just doesn't know? If you go where [I got the data](https://www.ntsb.gov/Pages/AviationQueryV2.aspx), there isn't a good answer. You'd probably have to contact the NTSB to find out what the difference is.

What about `AirportName`?

### Exercise 3: Missing data part 2

Use `.value_counts()` on the `AirportName` column.

In [ ]:
# Get counts of each unique value in 'AirportName'
airport_counts = crashes_df[____].value_counts(dropna=____)
print(airport_counts)

# Check the exercise
check_tally(airport_counts, 'AirportName')

What does `NaN` mean here? `NaN` will often be listed if you use `dropna=False`. Does it mean the NTSB doesn't know where this happened? Or does it mean the accident didn't happen at an airport? The answer here is that not all plane crashes happen at an airport ... but a lot of them do. So this isn't necessarily *missing* data in the sense of an error -- it's likely blank for a valid reason (no airport involved).

## Internal inconsistency

Any time you are going to focus on something, you should check it for consistency inside the data set. So let's pretend you want to look at *who makes* the airplanes that crash. To be clear -- this is a bad idea. There are only a small number of manufacturers and counting the number that crashed is no different than looking at cars that crash. Does a lot of Fords and Chevys crashing mean they're unsafe? Or does it mean there's a *lot* of Fords and Chevys on the road in the United States.

But, to illustrate the problem of internal consistency, we're going to do this. We're going to put the spotlight on the `Make` column.

### Exercise 4: Are they the same?

Use `.value_counts()` on the `Make` column to see the different manufacturer names recorded.

In [ ]:
# Get counts of each unique value in 'Make'
make_counts = crashes_df[____].value_counts(dropna=____)
print(make_counts)

# Check the exercise
check_tally(make_counts, 'Make')

Every state is going to be different here, but scroll through the results. Do you notice anything? For data to be exact, each row must be the same. And by the same I mean EXACTLY the same. For example: `CESSNA` and `Cessna` are not the same to a computer unless you tell it to ignore case. `Cirrus` and `CIRRUS DESIGN GROUP` are not the same. `Piper`, `PIPER` and `PIPER AIRCRAFT INC` are very much not the same thing in data world. Now, are they in real life? Yes. You and I can figure that out pretty quickly with just a tiny amount of brainpower. But data analysis requires things to be consistent and identical. And this column of data is not internally consistent.

Can you accurately report on the number of accidents involving each manufacturer? Not yet, but you can (more on that soon!).

And that's what Data Smells are designed to do: stop you from going down a bad path.

## The Recap

Throughout this lesson, you've learned how to apply the concept of "data smells" to critically examine a dataset using Python and pandas. You've practiced using `.info()` to check for wrong data types, `.value_counts()` (or `groupby().size()`) to identify missing data and gaps, and explored ways to spot internal inconsistencies in your data. Remember, these initial checks are crucial steps in any data analysis project. They help you understand the limitations of your data and prevent you from drawing incorrect conclusions. Approach new datasets with healthy skepticism and use these techniques to validate your data before proceeding with more in-depth analysis.

## Terms to Know

- **Data Smells**: Common issues or inconsistencies in datasets that may indicate potential problems.
- **Exploratory Data Analysis (EDA)**: An approach to analyzing datasets to summarize their main characteristics, often using visual methods and summary statistics.
- **Wrong Type of Data**: When data is imported or stored in an inappropriate format (e.g., dates stored as text/`object` instead of `datetime64[ns]`).
- **Missing Data**: Values that are absent from the dataset, often represented as `NaN` (Not a Number) in pandas.
- **Gaps in Data**: Unexpected breaks or discontinuities in a dataset, often relevant in time-series data.
- **Internal Inconsistency**: When the same information is represented differently within a dataset (e.g., inconsistent spelling or capitalization like `CESSNA` vs `Cessna`).
- **`.info()`**: A pandas DataFrame method that prints a concise summary of the DataFrame, including column data types (`Dtype`), non-null values, and memory usage.
- **`.value_counts()`**: A pandas Series method that returns a Series containing counts of unique values. Useful for checking consistency and finding missing values (with `dropna=False`).
- **`.groupby().size()`**: A pandas operation to group data by one or more columns and count the number of rows in each group.
- **`NaN` (Not a Number)**: NumPy's and pandas' standard representation for missing numerical data. Also used more broadly for missing values in pandas.
- **`object` dtype**: A pandas data type that often indicates strings, but can hold mixed types.
- **`datetime64[ns]`**: The typical pandas data type for datetime objects, representing nanosecond precision.